In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo 

# Equipo4 Integrantes
# Ariadne Selena Romero Rivero
# Bárbara Fernanda Cantú Reyes
# Diego Mauricio Alvarado Esparza
# Oziel Rebolledo Pérez

In [2]:
# Load data hoja 1
# Parametros casas nuevas
data_CM = pd.read_excel("data/Problema4.xlsx", sheet_name="Hoja1")
data_CM

,Casas_Modernas,Acres,Impuesto,Costo,Porcentaje
0,Unifamiliares,0.18,1000,50000,0.20
1,Dobles,0.28,1900,70000,0.10
2,Triples,0.40,2700,130000,0.25
3,Cuadruples,0.50,3400,160000,0.25


In [3]:
# Load data hoja 2
# Parametros casas viejas
data_CV = pd.read_excel("data/Problema4.xlsx", sheet_name="Hoja2")
data_CV

,Financiamiento,Casas_viejas_disponibles_a_demoler,Area_neta_disponible,Costo_por_demoler_cada_casa
0,15000000,300,0.2125,2000


In [4]:
# Para un ConcreteModel necesitamos los datos al inicio
# Conjuntos
I = data_CM["Casas_Modernas"].unique()  # Conjunto de casas nuevas a fabricar

# Parametros
G = data_CM.groupby(by=["Casas_Modernas"]).sum()[["Impuesto"]] #Impuesto por unidad de casa nueva i
A = data_CM.groupby(by=["Casas_Modernas"]).sum()[["Acres"]] #Cantidad de acres neto para casas nuevas i
C = data_CM.groupby(by=["Casas_Modernas"]).sum()[["Costo"]] # Costo por construir casa nueva i
Z = data_CM.groupby(by=["Casas_Modernas"]).sum()[["Porcentaje"]] # Porcentaje de casas de nuevas i

P = data_CV['Financiamiento'][0] # Financiamiento y/o Presupuesto = 15000000
J = data_CV['Casas_viejas_disponibles_a_demoler'][0] #Numero maximo de casas viejas disponibles para demoler = 300
B = data_CV['Area_neta_disponible'][0] # Area disponible luego de demoler una casa vieja = 0.2125
K = data_CV['Costo_por_demoler_cada_casa'][0] #Costo para demoler 1 casa vieja, esta la agregamos nosotros a la tabla = 2000


In [5]:
# Impresión de parametros :))
A

,Acres
Casas_Modernas,
Cuadruples,0.50
Dobles,0.28
Triples,0.40
Unifamiliares,0.18


# Concrete Model

In [6]:
# Creamos una variable de la clase ConcreteModel
model = pyo.ConcreteModel()

# Variables de decisión
model.x = pyo.Var(I, within=pyo.NonNegativeIntegers, doc="Cantidad de casas modernas i a construir")
model.w = pyo.Var(within=pyo.NonNegativeIntegers, doc="Cantidad de casas viejas a demoler")

# Función objetivo
model.obj = pyo.Objective(expr=sum(G.loc[i].values[0]*model.x[i] for i in I), sense=pyo.maximize)

# Restricciones:

# Restriccion 1: ---------------------------------------------------------------------------------
#Cantidad de casas a demoler (w) no supere la cantidad maxima de casas disponibles (J)
model.con1 = pyo.ConstraintList()
model.con1.add(expr=(model.w <= J))
#----------------------------------------------------------------------------------------------------


# Restriccion 2-------------------------------------------------------------------------------------
# Cantidad de acres neto por demoler casas se menor o igual a la cantidad de Acres  utilizados por las casas nuevas
model.con2 = pyo.ConstraintList()
model.con2.add(expr=sum(A.loc[i].values[0]*model.x[i] for i in I) <= B*model.w) #Falta la w jajan't
#----------------------------------------------------------------------------------------------------


#Restriccion 3-----------------------------------------------------------------------------------------
#El costo de construir casa nuevas y demoler casas viejas no exceda el financiamiento
model.con3 = pyo.ConstraintList()

def restriccion(model):
    return model.w * K + sum(C.loc[i].values[0] * model.x[i] for i in I) <= P

model.con3.add(restriccion(model)) # Agregar la restricción a la lista
#Lo hicimos asi porque de la otra manera nos daba error :cc
#--------------------------------------------- ----------------------------------------------------------


### Restrición 4
Dado que el problema nos menciona **"Las casas Triples y Cuadruples deben cumplir por lo menos el 25% de todas las casas nuevas"**.
Esta de forma matemática se expresa:

$$
x_3 + x_4 \geq 0.25 \sum_{i} x_i
$$

In [7]:
# Restriccion 4
# Las casas Triples y Cuadruples deben cumplir por lo menos el 25% de todas las casas nuevas
model.con4 = pyo.Constraint(expr=model.x[I[2]] + model.x[I[3]] >= 0.25 * sum(model.x[i] for i in I))

# Nota: No logramos sacar cada uno de los porcentajes del excel :(
# El 0.25 es el porcentaje aplicado a toda la restricción 4

### Restrición 5
Dado que el problema nos menciona **Las casas unifamiliares deben cumplir minimo el 20% de todas las casas nuevas**.
Esta de forma matemática se expresa:

$$
x_1 \geq 0.20 \sum_{i} x_i
$$

In [8]:
# Restriccion 5
# Las casas unifamiliares deben cumplir minimo el 20% de todas las casas nuevas
model.con5 = pyo.Constraint(expr=model.x[I[0]] >= 0.20 * sum(model.x[i] for i in I))

# El 0.20 es el porcentaje aplicado a toda la restricción 5

### Restrición 6
Dado que el problema nos menciona **Las casas dobles deben cumplir al menos el 10% de todas las casas nuevas**.
Esta de forma matemática se expresa:

$$
x_2 \geq 0.10 \sum_{i} x_i
$$

In [9]:
# Restriccion 6
# Las casas dobles deben cumplir al menos el 10% de todas las casas nuevas
model.con6 = pyo.Constraint(expr=model.x[I[1]] >= 0.10 * sum(model.x[i] for i in I))

# El 0.10 es el porcentaje aplicado a toda la restricción 6

In [10]:
model.pprint()

2 Var Declarations
    w : Cantidad de casas viejas a demoler
        Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x : Cantidad de casas modernas i a construir
        Size=4, Index={Cuadruples, Unifamiliares, Dobles, Triples}
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
           Cuadruples :     0 :  None :  None : False :  True : NonNegativeIntegers
               Dobles :     0 :  None :  None : False :  True : NonNegativeIntegers
              Triples :     0 :  None :  None : False :  True : NonNegativeIntegers
        Unifamiliares :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 1000*x[Unifamiliares] + 1900*x[Dobles] + 2700*x[Triples] + 3400*x[Cuadruples]

6 Constraint Declaratio

In [11]:
# Llamamos al solver GLPK y resolvemos el problema
solver = pyo.SolverFactory("glpk")
resultados = solver.solve(model)

In [12]:
resultados

{'Problem': [{'Name': 'unknown', 'Lower bound': 343700.0, 'Upper bound': 343700.0, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 5, 'Number of nonzeros': 23, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '11', 'Number of created subproblems': '11'}}, 'Error rc': 0, 'Time': 0.04245805740356445}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [13]:
for casa in I: 
    print("Cantidad de casas {} a construir: {}".format(casa, pyo.value(model.x[casa])))

Cantidad de casas Unifamiliares a construir: 36.0
Cantidad de casas Dobles a construir: 98.0
Cantidad de casas Triples a construir: 45.0
Cantidad de casas Cuadruples a construir: 0.0


In [14]:
#Cantidad de casas a demoler
print("La cantidad de casas demolidas es:", pyo.value(model.w))

La cantidad de casas demolidas es: 245.0


In [15]:
# Recaudación maxima del total de impuestos
pyo.value(model.obj)

343700.0